In [148]:
import pandas as pd
import numpy as np
import jsonlines
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import Normalizer

def load_jsonlines(path: str):
    transformer = Normalizer()
    data = []
    client_id_last = None
    with jsonlines.open(path, "r") as reader:
        for items in reader:
            #items["client_id"] = str(items["client_id"])
            if items["client_id"] == client_id_last:
                
                for i in range(len(items["amounts"])):
                    data[-1]["amounts"][items["transactions"][i]] += items["amounts"][i]
                
                for i in range(len(items["transactions"])):
                    items["transactions"][i] = str(items["transactions"][i])
                data[-1]["transactions"] += items["transactions"]
                #data[-1]["amounts"] += items["amounts"]
            else:
                if len(data) > 0:
                    data[-1]["amounts"] = transformer.transform(data[-1]["amounts"].reshape(1, -1))[0]
                data.append(items)
                data[-1]["amounts"] = np.zeros(500) #[0] * 500
                for i in range(len(items["transactions"])):
                    #print(items["amounts"][i])
                    data[-1]["amounts"][data[-1]["transactions"][i]] += items["amounts"][i]
                for i in range(len(items["transactions"])):
                    data[-1]["transactions"][i] = str(data[-1]["transactions"][i])
                
                #
            client_id_last = items["client_id"]
    data[-1]["amounts"] = transformer.transform(data[-1]["amounts"].reshape(1, -1))[0]
    return data

In [149]:
#!pip install jsonlines

In [150]:
X_train = load_jsonlines("substitute_clf/train.jsonl")
X_train = pd.DataFrame(X_train)
Y_train = X_train["label"]
del(X_train["label"])
X_train.head()

,transactions,amounts,client_id
0,"[259, 258, 259, 258, 259, 258, 222, 259, 259, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",144
1,"[215, 305, 215, 305, 215, 305, 215, 305, 215, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",794
2,"[172, 253, 233, 251, 155, 251, 210, 205, 260, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",830
3,"[210, 155, 215, 210, 210, 246, 215, 215, 210, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",936
4,"[330, 207, 210, 210, 220, 210, 210, 210, 251, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1197


In [151]:
len(X_train)

8387

In [152]:
X_train[:50]

,transactions,amounts,client_id
0,"[259, 258, 259, 258, 259, 258, 222, 259, 259, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",144
1,"[215, 305, 215, 305, 215, 305, 215, 305, 215, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",794
2,"[172, 253, 233, 251, 155, 251, 210, 205, 260, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",830
3,"[210, 155, 215, 210, 210, 246, 215, 215, 210, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",936
4,"[330, 207, 210, 210, 220, 210, 210, 210, 251, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1197
5,"[210, 236, 234, 210, 290, 290, 221, 290, 221, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1230
6,"[210, 210, 210, 210, 210, 253, 210, 251, 251, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1864
7,"[210, 259, 231, 207, 259, 210, 210, 210, 210, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1939
8,"[253, 251, 221, 210, 246, 246, 292, 251, 221, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1980
9,"[215, 210, 215, 233, 215, 215, 344, 259, 210, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2047


In [83]:
#X_train["transactions"][0] += X_train["transactions"][3]

In [153]:
Y_train.head()

0    2
1    2
2    2
3    2
4    1
Name: label, dtype: int64

In [154]:
Y_train.unique()

array([2, 1, 3, 0])

In [155]:
X_valid = load_jsonlines("substitute_clf/valid.jsonl")
X_valid = pd.DataFrame(X_valid)
Y_valid = X_valid["label"]
del(X_valid["label"])
X_valid.head()

,transactions,amounts,client_id
0,"[215, 300, 198, 215, 300, 198, 215, 300, 300, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",183
1,"[155, 259, 259, 155, 210, 253, 210, 259, 155, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",629
2,"[221, 251, 259, 220, 253, 210, 249, 258, 258, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",882
3,"[251, 251, 221, 210, 210, 198, 258, 251, 210, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",928
4,"[155, 155, 155, 170, 341, 209, 255, 169, 169, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1156


In [88]:
X_test = load_jsonlines("test.jsonl")
X_test = pd.DataFrame(X_train)
X_test.head()

,transactions,amounts,client_id
0,"[259, 258, 259, 258, 259, 258, 222, 259, 259, ...","[0, 3000, 0, 3000, 0, 3000, 1000, 500, 0, 3000...",144
1,"[215, 305, 215, 305, 215, 305, 215, 305, 215, ...","[2000, 2000, 2000, 2000, 2000, 2000, 2000, 200...",794
2,"[172, 253, 233, 251, 155, 251, 210, 205, 260, ...","[500, 0, 8000, 500, 500, 1000, 500, 1000, 500,...",830
3,"[210, 155, 215, 210, 210, 246, 215, 215, 210, ...","[500, 500, 0, 500, 500, 500, 500, 500, 500, 50...",936
4,"[330, 207, 210, 210, 220, 210, 210, 210, 251, ...","[500, 500, 1000, 0, 500, 500, 500, 3000, 2000,...",1197


In [156]:
from sklearn.feature_extraction.text import TfidfTransformer

In [187]:
# working about 3 min on train
# working with one column as X_train.transactions or X_train.amounts

tf = X_train.transactions.apply(lambda x: pd.Series(x).value_counts()).fillna(0)   
tf.sort_index(inplace=True, axis=1)
print("Find these keys:", tf.keys())
    
tfidf_transformer = TfidfTransformer()
X = tfidf_transformer.fit_transform(tf)
idf = pd.DataFrame({'feature_name':tf.keys(), 'idf_weights':tfidf_transformer.idf_})
print("idf: ", idf)

tf_idf = pd.DataFrame(X.toarray() ,columns=tf.keys())
print("tf_idf:", tf_idf)


Find these keys: Index(['0', '1', '10', '100', '101', '103', '105', '108', '109', '111',
       ...
       '84', '85', '87', '89', '9', '93', '94', '95', '98', '99'],
      dtype='object', length=326)
idf:      feature_name  idf_weights
0              0     8.935945
1              1     7.731972
2             10     9.341410
3            100     9.341410
4            101     8.935945
..           ...          ...
321           93     9.341410
322           94     9.341410
323           95     9.341410
324           98     9.341410
325           99     7.837333

[326 rows x 2 columns]
tf_idf:         0    1   10  100  101  103  105  108  109  111  ...   84   85   87  \
0     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
1     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
3     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
4     0.0  

In [237]:
X = tf_idf

In [231]:
#temp = np.array(X)
temp.shape

(8387, 500)

In [219]:
X_train["amounts"][0].shape

(500,)

In [210]:
np.array(X_train["amounts"]).shape

(8387,)

In [240]:
X_final = np.concatenate((X, temp), axis=1)

In [243]:
X_final.shape

(8387, 826)

In [239]:
temp = np.zeros((len(X_train["amounts"]), 500))
for i in range(len(temp)):
    temp[i] = X_train["amounts"][i]
X_final = np.concatenate((X, temp), axis=1)    
    
    #        X[key][i] = np.array([X[key][i], X_train["amounts"][i][int(key)]])#*= X_train["amounts"][i][int(key)]
    #    else:
    #        X[key][i] = np.array([0., 0.])

In [185]:
np.array([X["259"][0], X_train["amounts"][0][259]])

0.0290892632377609

In [161]:
X['259']

0       0.029089
1       0.000000
2       0.001634
3       0.000000
4       0.000000
          ...   
8382    0.198469
8383    0.000000
8384    0.005138
8385    0.000000
8386    0.000000
Name: 259, Length: 8387, dtype: float64

In [93]:
tf = X_train.amounts.apply(lambda x: pd.Series(x).value_counts()).fillna(0)   
tf.sort_index(inplace=True, axis=1)
print("Find these keys:", tf.keys())
    
tfidf_transformer_amounts = TfidfTransformer()
X1 = tfidf_transformer_amounts.fit_transform(tf)
idf = pd.DataFrame({'feature_name':tf.keys(), 'idf_weights':tfidf_transformer_amounts.idf_})
print("idf: ", idf)

tf_idf = pd.DataFrame(X1.toarray() ,columns=tf.keys())
print("tf_idf:", tf_idf)

Find these keys: Index(['0', '1000', '10000', '100000', '100500', '101000', '101500', '102500',
       '103000', '103500',
       ...
       '9500', '95000', '95500', '96500', '97000', '97500', '98000', '98500',
       '99000', '99500'],
      dtype='object', length=321)
idf:      feature_name  idf_weights
0              0     1.074989
1           1000     1.183180
2          10000     3.661238
3         100000     7.326507
4         100500     8.648263
..           ...          ...
316        97500     8.425119
317        98000     9.341410
318        98500     8.088647
319        99000     8.935945
320        99500     8.242798

[321 rows x 2 columns]
tf_idf:              0      1000     10000  100000  100500  101000  101500  102500  \
0     0.621170  0.097670  0.000000     0.0     0.0     0.0     0.0     0.0   
1     0.000000  0.000000  0.000000     0.0     0.0     0.0     0.0     0.0   
2     0.388005  0.375148  0.000000     0.0     0.0     0.0     0.0     0.0   
3     0.879371  0.

In [94]:
X1 = tf_idf
X1

,0,1000,10000,100000,100500,101000,101500,102500,103000,103500,...,9500,95000,95500,96500,97000,97500,98000,98500,99000,99500
0,0.621170,0.097670,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.388005,0.375148,0.000000,0.0,0.0,0.0,0.0,0.0,0.178195,0.0,...,0.118428,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.879371,0.090547,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.846406,0.077492,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8382,0.881299,0.272126,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8383,0.125271,0.080429,0.308138,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8384,0.694509,0.278281,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8385,0.472387,0.230967,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [95]:
X["777"] = X["0"]
del(X["0"])

In [96]:
import pandas as pd
X_final = pd.concat([X, X1], sort = False, axis = 1)

In [162]:
X_final = X

In [97]:
#X_final_final = pd.concat([X_final, X_train["label"]], sort = False, axis = 1)

In [163]:
tf = X_valid.transactions.apply(lambda x: pd.Series(x).value_counts()).fillna(0)   
tf.sort_index(inplace=True, axis=1)
print("Find these keys:", tf.keys())
tf = pd.DataFrame(tf, columns=X.keys()).fillna(0)  # yes, here we need X.keys from train fitting 
 
X2 = tfidf_transformer.transform(tf)
idf = pd.DataFrame({'feature_name':tf.keys(), 'idf_weights':tfidf_transformer.idf_})
print("idf: ", idf)

tf_idf = pd.DataFrame(X2.toarray() ,columns=tf.keys())
print("tf_idf:", tf_idf)

Find these keys: Index(['0', '1', '103', '11', '111', '122', '123', '127', '129', '13',
       ...
       '75', '76', '79', '8', '81', '84', '86', '87', '92', '99'],
      dtype='object', length=260)
idf:      feature_name  idf_weights
0              0     8.935945
1              1     7.731972
2             10     9.341410
3            100     9.341410
4            101     8.935945
..           ...          ...
321           93     9.341410
322           94     9.341410
323           95     9.341410
324           98     9.341410
325           99     7.837333

[326 rows x 2 columns]
tf_idf:         0    1   10  100  101  103  105  108  109  111  ...   84   85   87  \
0     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
1     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
3     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
4     0.0  0

In [164]:
X2 = tf_idf
X2

,0,1,10,100,101,103,105,108,109,111,...,84,85,87,89,9,93,94,95,98,99
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2115,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2116,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [247]:
temp = np.zeros((len(X_valid["amounts"]), 500))
for i in range(len(temp)):
    temp[i] = X_valid["amounts"][i]
X_valid_final = np.concatenate((X2, temp), axis=1)    
  

#for key in X2.keys():
#    for i in range(len(X2)):
#        if X2[key][i] > 0.0:
#            X2[key][i] *= X_valid["amounts"][i][int(key)]


In [166]:
X_valid_final = X2

In [ ]:
tf = X_valid.amounts.apply(lambda x: pd.Series(x).value_counts()).fillna(0)   
tf.sort_index(inplace=True, axis=1)
print("Find these keys:", tf.keys())
tf = pd.DataFrame(tf, columns=X1.keys()).fillna(0)  # yes, here we need X2.keys() from train fitting

#tfidf_transformer_3 = TfidfTransformer()
X3 = tfidf_transformer_amounts.transform(tf)
idf = pd.DataFrame({'feature_name':tf.keys(), 'idf_weights':tfidf_transformer_amounts.idf_})
print("idf: ", idf)

tf_idf = pd.DataFrame(X3.toarray() ,columns=tf.keys())
print("tf_idf:", tf_idf)

In [103]:
X3 = tf_idf
X3

,0,1,10,100,101,103,105,108,109,111,...,84,85,87,89,9,93,94,95,98,99
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2115,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2116,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [104]:
#X2["777"] = X2["0"]
#del(X2["0"])

In [105]:
import pandas as pd
X_valid_final = pd.concat([X2, X3], sort = False, axis = 1)

In [167]:
X_valid_final["1000"].max()

KeyError: '1000'

In [168]:
X_valid_final.max()

0      0.004308
1      0.179523
10     0.000000
100    0.000000
101    0.000000
         ...   
93     0.000000
94     0.000000
95     0.000000
98     0.000000
99     0.104801
Length: 326, dtype: float64

In [244]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn import svm, datasets
pipe = Pipeline([('classifier' , LogisticRegression())])
# pipe = Pipeline([('classifier', RandomForestClassifier())])

# Create param grid.

param_grid = [
    {'classifier' : [LogisticRegression()],
     'classifier__penalty' : ['l1', 'l2'],
    'classifier__C' : np.logspace(-4, 4, 10),
    'classifier__solver' : ['liblinear']},
]

# Create grid search object
"""
clf = GridSearchCV(pipe, param_grid = param_grid, cv = 5, verbose=True, n_jobs=-1)

# Fit on data

best_clf = clf.fit(X, Y_train)
print(clf.score(X, Y_train))
"""

#pipe = Pipeline('lr', LogisticRegression(multi_class='multinomial', tol=1e-3, max_iter=500)
clf = GridSearchCV(pipe, param_grid = param_grid, cv = 5, verbose=True, n_jobs=3)
clf.fit(X_final, Y_train)
clf_pred = clf.predict(X_final)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  67 tasks      | elapsed:    4.4s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:  1.1min finished


In [245]:
import joblib

print(clf.score(X_final, Y_train))

#save your model or results
joblib.dump(clf, 'np_data_normal_logistic_cv_5_[-4,4,10].pkl')

#load your model for further usage
#joblib.load("model_file_name.pkl")


0.6740193156074877


['np_data_normal_logistic_cv_5_[-4,4,10].pkl']

In [248]:
clf.score(X_valid_final, Y_valid)

0.6383380547686497

In [249]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn import svm, datasets
##pipe = Pipeline([('classifier' , LogisticRegression())])
"""
pipe = Pipeline([('classifier', RandomForestClassifier(max_depth=17, random_state=0))])

# Create param grid.

param_grid = [
    {'classifier' : [LogisticRegression()],
     'classifier__penalty' : ['l1', 'l2'],
    'classifier__C' : np.logspace(1, 1, 1),
    'classifier__solver' : ['liblinear']},
]

# Create grid search object
clf = GridSearchCV(pipe, param_grid = param_grid, cv = 5, verbose=True, n_jobs=-1)

# Fit on data

best_clf = clf.fit(X, Y_train)
print(clf.score(X, Y_train))
"""
from sklearn.ensemble import RandomForestClassifier
 
clf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=50, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)
clf.fit(X_final, Y_train)

"""
pipe = Pipeline('lr', LogisticRegression(multi_class='multinomial', tol=1e-3, max_iter=500)
clf = GridSearchCV(pipe, param_grid = param_grid, cv = 5, verbose=True, n_jobs=-1)
clf.fit(X_final, Y_train)
clf_pred = clf.predict(X_final)
"""

"\npipe = Pipeline('lr', LogisticRegression(multi_class='multinomial', tol=1e-3, max_iter=500)\nclf = GridSearchCV(pipe, param_grid = param_grid, cv = 5, verbose=True, n_jobs=-1)\nclf.fit(X_final, Y_train)\nclf_pred = clf.predict(X_final)\n"

In [250]:
print(clf.score(X_final, Y_train), " train acc")
print(clf.score(X_valid_final, Y_valid), " valid acc")
#save your model or results
joblib.dump(clf, 'np_data_normal_forest_dep_50_est_100.pkl')


0.9717419816382497  train acc
0.6586402266288952  valid acc


['np_data_normal_forest_dep_50_est_100.pkl']

In [257]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=16, learning_rate=0.1,
max_depth=10, random_state=0).fit(X_final, Y_train)


In [258]:
print(clf.score(X_final, Y_train), " train acc")
print(clf.score(X_valid_final, Y_valid), " valid acc")
#save your model or results
joblib.dump(clf, 'np_data_normal_BOOSTING_dep_10_est_16_lr_0.1.pkl')

0.8399904614284011  train acc
0.653918791312559  valid acc


['np_data_normal_BOOSTING_dep_10_est_16_lr_0.1.pkl']

In [259]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_final, Y_train)
#Pipeline(steps=[('standardscaler', StandardScaler()),
#                ('svc', SVC(gamma='auto'))])

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [260]:
joblib.dump(clf, 'np_data_normal_SVC_1.pkl')

['np_data_normal_SVC_1.pkl']

In [261]:
print(clf.score(X_final, Y_train), " train acc")
print(clf.score(X_valid_final, Y_valid), " valid acc")
#save your model or results
joblib.dump(clf, 'np_data_normal_SVC_1.pkl')

0.6888041015857875  train acc
0.6581680830972616  valid acc


['np_data_normal_SVC_1.pkl']

In [ ]:
tf = X_test.transactions.apply(lambda x: pd.Series(x).value_counts()).fillna(0)   
tf.sort_index(inplace=True, axis=1)
print("Find these keys:", tf.keys())
tf = pd.DataFrame(tf, columns=X.keys()).fillna(0)  # yes, here we need X.keys from train fitting 
 
X_testing = tfidf_transformer.transform(tf)
idf = pd.DataFrame({'feature_name':tf.keys(), 'idf_weights':tfidf_transformer.idf_})
print("idf: ", idf)

tf_idf = pd.DataFrame(X_testing.toarray() ,columns=tf.keys())
print("tf_idf:", tf_idf)

In [ ]:
X_testing = tf_idf
X_testing

In [ ]:
tf = X_test.amounts.apply(lambda x: pd.Series(x).value_counts()).fillna(0)   
tf.sort_index(inplace=True, axis=1)
print("Find these keys:", tf.keys())
tf = pd.DataFrame(tf, columns=X1.keys()).fillna(0)  # yes, here we need X.keys from train fitting 
 
X_testing_1 = tfidf_transformer.transform(tf)
idf = pd.DataFrame({'feature_name':tf.keys(), 'idf_weights':tfidf_transformer.idf_})
print("idf: ", idf)

tf_idf = pd.DataFrame(X_testing_1.toarray() ,columns=tf.keys())
print("tf_idf:", tf_idf)

In [ ]:
X_testing_1 = tf_idf
X_testing_1

In [ ]:
X_testing["777"] = X_testing["0"]
del(X_testing["0"])